In [ ]:
##We load the packages used at the work
##We are going to use as rule pd for Pandas
##If you are going to use Matplotlib, Numpy or
##other, you should search how to name them 
## as rule.
##For keep the order you can use prettier
## in vscode


##Pandas is similar to tidyverse
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
##The next package is used to format tables
##In the next lines you can watch how to use it
from tabulate import tabulate
from sklearn import svm
from sklearn.metrics import roc_curve, auc

##La tabla que vamos a emplear es esta, las demás han
##sido eliminadas
file="SouthGermanCredit.asc"
data=pd.read_csv(file," ")

##We are going to change the name of the variables 
##because in German is difficult to us work.
names={'laufkont':'Status',
        'laufzeit':'Duracion',
        'moral':'Historial_Crediticio',
        'verw':'Proposito',
        'hoehe':'Monto',
        'sparkont':'Ahorros',
        'beszeit':'Tiempo_trabajando',
        'rate':'Porcentaje_salario',
        'famges':'Estado_civil_sexo',
        'buerge':'Otros_deudores',
        'wohnzeit':'Residencia',
        'verm':'Propiedad_valiosa',
        'alter':'Edad',
        'weitkred':'Otras_deudas',
        'wohn':'Residencia',
        'bishkred':'Creditos_banco',
        'beruf':'Empleo',
        'pers':'Personas_dependientes',
        'telef':'Telefono',
        'gastarb':'Extranjero',
        'kredit':'Default',}


data.rename(columns=names,inplace=True)
print(data.head(5))
#print(tabulate(data.head(5),headers='keys',tablefmt='latex_longtable'))


In [ ]:
##It's contructed a table of min and max of all to know if 
##the data doesn't have mistakes
data_1=data.agg(['min','max'])
print(tabulate(data_1.transpose(),headers='keys',tablefmt='latex_longtable'))

In [ ]:
##We are going to construct the tables
##We are going to do a resume of five numbers
##Duracion, Monto, Edad
##First we select this three columns
##This new variable is the data.frame with,
##what we want
data_2=data[['Duracion','Edad','Monto','Tiempo_trabajando']]

summary_2=data_2.describe().transpose()
summary_2=summary_2.loc[:,summary_2.columns!='count']



print(tabulate(summary_2,headers='keys',tablefmt='latex_longtable'))

In [ ]:
#We estimate the quartile coefficient of dispersion
summary_2['Coeficiente de dispersion cuartil']=(summary_2['75%']-summary_2['25%'])/(summary_2['75%']+summary_2['25%'])
summary_2['Coeficiente de variacion']=summary_2['std']/summary_2['mean']

print(tabulate(summary_2[['Coeficiente de dispersion cuartial','Coeficiente de variacion']],headers='keys',floatfmt=".4f",tablefmt='latex_longtable'))

In [ ]:
##A table count historial and default
historial_default=pd.crosstab(index=data['Default'],columns=data['Historial_Crediticio'],margins=True)
print(tabulate(historial_default,headers='keys',tablefmt='latex_longtable'))

In [ ]:
##A table count savings and default
savings_default=pd.crosstab(index=data['Default'],columns=data['Ahorros'])
print(tabulate(savings_default,headers='keys',tablefmt='latex_longtable'))

In [11]:
#Before we start we are going to change the model form 0 to -1
data.loc[data.Default==0,'Default']=-1

#Here we are going to construct the way in wich we construct the sample
#Neverthless what we want to make to this is a model that at this moment have not been programmed so we are going to select the sample dividing the table in two the good and the bads 
#In this case we are going to select 80% of the good and bads, and also for
#20% of training
goods=data[(data['Default']==1)]
bads=data[(data['Default']==-1)]
#With this we construct the sample

goods_random=goods.sample(n=round(len(goods)*0.8),replace=False)
bads_random=bads.sample(n=round(len(bads)*0.8),replace=False)
train=pd.concat([goods_random,bads_random])
test_random=data.drop(train.index)
bads_random


,Status,Duracion,Historial_Crediticio,Proposito,Monto,Ahorros,Tiempo_trabajando,Porcentaje_salario,Estado_civil_sexo,Otros_deudores,...,Propiedad_valiosa,Edad,Otras_deudas,Residencia,Creditos_banco,Empleo,Personas_dependientes,Telefono,Extranjero,Default
922,1,24,2,0,1442,1,4,4,2,1,...,3,23,3,1,2,3,2,1,2,-1
855,1,48,2,0,3931,1,4,4,3,1,...,4,46,3,3,1,3,1,1,2,-1
845,2,27,4,3,2520,3,3,4,3,1,...,2,23,3,2,2,2,2,1,2,-1
838,4,18,1,0,6458,1,5,2,3,1,...,4,39,1,2,2,4,1,2,2,-1
917,2,12,0,2,2969,1,2,4,2,1,...,2,25,3,1,2,3,2,1,2,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,1,24,2,3,1823,1,1,4,3,1,...,3,30,2,2,1,4,1,1,2,-1
927,1,12,2,8,902,1,4,4,4,1,...,2,21,3,1,1,3,2,1,2,-1
920,1,24,1,2,3552,1,4,3,3,1,...,3,27,1,2,1,3,2,1,2,-1
952,2,21,2,9,1188,1,5,2,2,1,...,2,39,3,2,1,3,1,1,2,-1


In [ ]:
#Here we are going to program the svm, but with the four basic kernels,
#and with the same gamma
#We are going to keep the basic adjustment for in this case
#And we compare also it with the proposed by Ben-David and Shalev

##Linear, This is not accomplised for our data set
#linear_svm=svm.SVC(kernel='linear',probability=True)
#linear_svm.fit(train.drop('Default',1),train[['Default']])

##Polynomial of grade 2
poly_svm=svm.SVC(kernel='poly',degree=2,probability=True)
poly_svm.fit(train.drop('Default',1),train[['Default']])



##Gaussian Kernel
gauss_svm=svm.SVC(kernel='rbf',probability=True)
gauss_svm.fit(train.drop('Default',1),train[['Default']])

##sigmoid
sigm_svm=svm.SVC(kernel='sigmoid',probability=True)
sigm_svm.fit(train.drop('Default',1),train[['Default']])

In [ ]:
# We are going to construct the model here based in the model
#presented by Ben-David and Shalev
##Polynomial of grade 2
poly_svm_bd=svm.SVC(kernel='poly',degree=2,probability=True,C=1/len(test_random))
poly_svm_bd.fit(train.drop('Default',1),train[['Default']])



##Gaussian Kernel
gauss_svm_bd=svm.SVC(kernel='rbf',probability=True,C=1/len(test_random))
gauss_svm.fit(train.drop('Default',1),train[['Default']])

##sigmoid
sigm_svm_bd=svm.SVC(kernel='sigmoid',probability=True,C=1/len(test_random))
sigm_svm_bd.fit(train.drop('Default',1),train[['Default']])

